In [ ]:
from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

import re

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select

# Multi
from multiprocessing import Pool, cpu_count

# Time
import datetime
import time

# Tools
from itertools import repeat
import unittest
from unittest.mock import patch

# Bekeautocenter : scrapping


In [ ]:
# -------------------------------
# ---------- FONCTIONS ----------
# -------------------------------

next_page_button_xpath = '/html/body/div[2]/div/div[1]/form/div[2]/div[2]/div[2]/div/ul/li[7]/a'

def turn_page ():
    try:
        next_page_button = browser.find_element_by_xpath(next_page_button_xpath)
        next_page_button.click()
    except(NoSuchElementException):
        print ("Il n'y a plus de pages à tourner")
        
# dans la page de n resultats, recuperer les liens de chaque résultat -> liste de liens
def get_result_urls(sleep_time, resultats_par_page):
    url_list = []
    counter = 0
    for index in range(3,resultats_par_page+2):
        try:
            time.sleep(sleep_time)
            url_list.append(browser.find_element_by_xpath(f'/html/body/div[2]/div/div[1]/form/div[2]/div[{index}]/article/div/div[1]/div/a').get_attribute('href'))
            time.sleep(sleep_time)
            counter += 1
        except:
            print("Il n'y a plus d'url à récupérér!!!! Itérations effectuées : ", counter)
            break
    return url_list

# ouvrir url dans nouvelle fenêtre, executer get_attributes, remplir liste, fermer onglet -> liste complétée
def open_url_get_attributtes_go_back(sleep_time,liste_a_completer):
    counter = 0
    counter_small = 0
    for url in url_list:
        try:
            main_window = browser.current_window_handle
            # Open a new tab in blank
            browser.execute_script("window.open(''),'_blannk'")
            # Switch to the new window
            browser.switch_to.window(browser.window_handles[1])

            # Change the url in the .get**
            browser.get(url)
            # Completer la liste
            attributs_voiture = get_attributes(beke_occasion_xpath)
            liste_a_completer.append(attributs_voiture)
            # Temps d'attente
            time.sleep(sleep_time)
            # Close Current Tab
            browser.close()
            # Focus to the main window
            browser.switch_to.window(main_window)
            counter += 1
            counter_small += 1
            print(counter_small)
        
        except:
            print("Il n'y a plus d'url dans la liste. J'ai fini ! Itérations : ", counter)
            break
        
    return liste_a_completer

def get_attributes(dictionnaire):
    liste = []
    for item in dictionnaire.items() : 

                try:
                     liste.append(browser.find_element_by_xpath(item[1]).text)

                except (NoSuchElementException) : 
                    liste.append('nan')
    return liste

# VOITURES D'OCCASION SI OUVERT DANS NOUVEL L'ONGLET 
beke_occasion_xpath = {
    "nom":'/html/body/div[1]/div[1]/div/div/div[2]/h3',    
    "sous_nom":'/html/body/div[1]/div[1]/div/div/div[2]/span',
    "ref":"/html/body/div[1]/div[1]/div/div/div[3]/div[1]/div[2]/div",
    "marque":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[1]/a',
    "modele":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[2]/a',
    "categorie":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[3]/a',
    "co2":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]/div/div[2]',    
    "nb_km":"//td[preceding::th[text()[contains(.,'Kilométrage')]]]",    
    "carburant":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[1]/div[2]/span',    
    "boite_vitesse":"//span[preceding::label[text()[contains(.,'Boite')]]]",    
    "couleur":"//td[preceding::th[text()[contains(.,'Couleur')]]]",
    "nb_portes":"//td[preceding::th[text()[contains(.,'Nombre de portes')]]]",
    "nb_places":"//td[preceding::th[text()[contains(.,'Nombre de places')]]]",
    "mise_circulation":"//td[preceding::th[text()[contains(.,'circulation')]]]",    
    "conso_mixte":"//td[preceding::th[text()[contains(.,'Conso. mixte')]]]",    
    "p_fiscale":"//td[preceding::th[text()[contains(.,'Puissance fiscale')]]]",    
    "p_din":"//td[preceding::th[text()[contains(.,'Puissance DIN')]]]",    
    "critair":"//th[text()[contains(.,'Certificat')]]",    
    "prix":'/html/body/div[1]/div[1]/div/aside/div/div[1]/div[1]/div[1]/div/span',     
}

In [ ]:
%%time
# -------------------------------
# ---------- SCRAPPING ----------
# -------------------------------

# Recherche Renault, Citroen, Peugeot, 1àà resultats par page
url_beke = 'http://www.bekeautocenter.fr/catalogue/?search[vnvo]=1&search[marques][0]=Citro%C3%ABn&search[marques][1]=Peugeot&search[marques][2]=Renault&search[minPrice]=0&search[maxPrice]=40000&search[order]=marque|asc&search[limit]=100&search[_token]=S90FBTV8BwR2bWGAgT4fdtttoPj4ZPTKrUYdM27skPg&_=1590561915186&page=1'

# HEADLESS
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get(url_beke)



# HEAD-ON
# browser = webdriver.Chrome()
# browser.maximize_window()
# browser.get(url_beke)



# Nombre total de résultats au format : '816 RÉFÉRENCES' (type = str)
beke_total_results = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/form/div[1]/div[2]/h3').text
print("Nombre total de résultats :", beke_total_results)

# Dernière page disponible au format : 9 (type = int)
beke_last_page = int(re.findall("(\d+) ",beke_total_results)[0]) // 100 + 1
print("Dernière page disponible : ", beke_last_page)

liste_a_completer = []

for page in range(1,beke_last_page+1):
    
    # Liste de liens de la page en cours (sleep_time, resultats/page)
    url_list = get_result_urls(0.1, 100)

    # Va dans chaque lien et recupere les infos de chaque voiture (sleep_time, liste_a_completer)
    liste_a_completer = open_url_get_attributtes_go_back(1,liste_a_completer)

    # On tourne la page :
    turn_page() 
    
    print("Echantillons recupérés : ", len(liste_a_completer))
    


# Preprocess

In [ ]:
# Générer le pandas DataFrame
column_names = [item[0] for item in beke_occasion_xpath.items()]
raw = pd.DataFrame.from_records(liste_a_completer,columns=column_names)

In [ ]:
# Sauver le CSV brut
raw.to_csv("beke_raw.csv", index=False)

In [ ]:
# -----------------------------------------------------
# ------------- PREPROCESSING - FONCTIONS -------------
# -----------------------------------------------------

# GENERAL
def preprocess_beke_dataframe (df):
    
    print('initial : ', df.shape)   
    
    # Enlever les espaces entre chiffres
    df['prix'] = df['prix'].str.replace(' ', '', regex=True)
    df['p_din'] = df['p_din'].str.replace(' ', '', regex=True)
    df['ref'] = df['ref'].str.replace(' - ', '', regex=True)
    df['nb_km'] = df['nb_km'].str.replace(' ', '', regex=True)
    
    # Remplacer nan par '0' pour pouvoir convertir en nombre plus tard
    df['conso_mixte'] = df['conso_mixte'].str.replace('nan', '0.00L', regex=True)
    df['co2'] = df['co2'].str.replace('nan', '0g', regex=True)

    # Extraire par regex puis changer le type
    for col in ['nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix', 'co2']:
        df[col] = df[col].str.extract(r'(\d+)').astype(int)

    # Extraire par regex puis changer le type (colonnes particulières)
    df['conso_mixte'] = df['conso_mixte'].str.extract('(\d+\.?\d? ?)L').astype(float)
    df['ref'] = df['ref'].str.extract('(\d+)')
    df['nb_km'] = df['nb_km'].str.extract('(\d+ ?\d*)').astype(int)

    # Garder l'année seulement
    df['mise_circulation'] = df['mise_circulation'].str.extract('(\d{4})').astype(int)
    
    # Supprimer les doublons
    df.drop_duplicates(subset ="ref",keep = 'first', inplace = True, ignore_index = True)
    print('sans doublons : ', df.shape)
    
    # Supprimer les colonnes inutiles
    df = df.drop(['sous_nom'], axis=1)
    print('sans colonnes inutiles : ', df.shape)

    return df

# LABEL REJET CO2
def label_co2(df,col):
    
    for i in range(len(df[col])):
        
        if df.loc[i,col] <= 100: df.at[i, col] = 1
        elif df.loc[i,col] <= 120: df.at[i, col] = 2
        elif df.loc[i,col] <= 140: df.at[i, col] = 3
        elif df.loc[i,col] <= 160: df.at[i, col] = 4 
        elif df.loc[i,col] <= 200: df.at[i, col] = 5 
        else : df.at[i, col] = 6
        
    df[col] = df[col].astype(str)
    
    labels = {
        "1": "A",
        "2": "B",
        "3": "C",
        "4": "D",
        "5": "E",
        "6": "F"
    }
    
    for item in labels.items():
        df[col] = df[col].str.replace(item[0], item[1], regex=True)
        
    return df     

# ENLEVER LIGNES
def clean_row_by_value(df, col, drop_list):
    
    for name in drop_list:
        df = df[df[col] != name]
    print('sans ', drop_list,  df.shape)
    
    return df

# Seulement première lettre en majuscule
def first_letter_capital(df, numerical_columns):
    for i in [x for x in df.columns if x not in numerical_columns]:
        df[i] = df[i].str.lower()

    for i in [x for x in df.columns if x not in numerical_columns]:
        df[i] = df[i].str.capitalize() 
        
    return df

In [ ]:
# -------------------------------------------------------
# ------------- PREPROCESSING - APPLICATION -------------
# -------------------------------------------------------

raw = pd.DataFrame.from_records(liste_a_completer,columns=column_names)

# Général
df = preprocess_beke_dataframe(raw)

# Label CO2
df = label_co2(df, "co2")

# Enlever lignes
drop_list = ['UTILITAIRE', 'DIVERS', 'BREAK']
df = clean_row_by_value(df, "categorie", drop_list)

# Seulement première lettre en majuscule
numerical_columns = ['nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix', 'mise_circulation', 'conso_mixte']
first_letter_capital(df, numerical_columns)


In [ ]:
# Sauver le csv final
df.to_csv("beke_processed.csv", index=False)